In [29]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [30]:
Movies = pd.read_sql('SELECT * FROM movie_data', con=mysql)

In [31]:
Movies = Movies[Movies.revenue >= 1000]

In [32]:
Movies[['year', 'month', 'day']] = Movies['release_date'].str.split('-', expand=True)
Movies['year'] = Movies['year'].str[2:]

In [33]:
Movies['year'] = pd.to_numeric(Movies['year'], errors='coerce')
Movies['month'] = pd.to_numeric(Movies['month'], errors='coerce')

In [34]:
Movies = Movies.drop(['title','genres3','release_date','day'], axis = 1)

In [35]:
Movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3507 entries, 0 to 8452
Data columns (total 24 columns):
vote_average             3507 non-null float64
cast1                    3507 non-null float64
cast2                    3507 non-null float64
cast3                    3502 non-null float64
producer1                3494 non-null float64
producer2                3336 non-null float64
producer3                3024 non-null float64
original_language1       3507 non-null object
vote_count               3507 non-null int64
production_companies1    3507 non-null float64
production_companies2    2527 non-null float64
production_companies3    1520 non-null float64
director1                3507 non-null float64
director2                224 non-null float64
director3                48 non-null float64
revenue                  3507 non-null int64
genres1                  3507 non-null object
genres2                  2924 non-null object
production_countries1    3507 non-null object
production_

In [36]:
original_language1_dmyfull = pd.get_dummies(Movies['original_language1'], drop_first=True)
genres1_dmyfull = pd.get_dummies(Movies['genres1'], drop_first=True)
genres2_dmyfull = pd.get_dummies(Movies['genres2'], drop_first=True)
production_countries1_dmyfull = pd.get_dummies(Movies['production_countries1'], drop_first=True)
production_countries2_dmyfull = pd.get_dummies(Movies['production_countries2'], drop_first=True)
month_dmyfull = pd.get_dummies(Movies['month'], drop_first=True)
cast1_dmyfull = pd.get_dummies(Movies['cast1'], drop_first=True)
cast2_dmyfull = pd.get_dummies(Movies['cast2'], drop_first=True)
cast3_dmyfull = pd.get_dummies(Movies['cast3'], drop_first=True)
#producer1_dmyfull = pd.get_dummies(Movies['producer1'], drop_first=True)
#producer2_dmyfull = pd.get_dummies(Movies['producer2'], drop_first=True)
#producer3_dmyfull = pd.get_dummies(Movies['producer3'], drop_first=True)
#director1_dmyfull = pd.get_dummies(Movies['director1'], drop_first=True)
#director2_dmyfull = pd.get_dummies(Movies['director2'], drop_first=True)
#director3_dmyfull = pd.get_dummies(Movies['director3'], drop_first=True)
#writer1_dmyfull = pd.get_dummies(Movies['writer1'], drop_first=True)

In [37]:
# Concatenate dummy variables to data set 
#Movies_full = pd.concat([Movies,original_language1_dmyfull,production_countries1_dmyfull,production_countries2_dmyfull,
 #                        genres1_dmyfull,genres2_dmyfull,cast1_dmyfull,cast2_dmyfull,cast3_dmyfull,producer1_dmyfull,
  #                      producer2_dmyfull,producer3_dmyfull,director1_dmyfull,director2_dmyfull,
   #                     director3_dmyfull,writer1_dmyfull], axis = 1 )
Movies_full = pd.concat([Movies,original_language1_dmyfull,production_countries1_dmyfull,production_countries2_dmyfull,
                         genres1_dmyfull,genres2_dmyfull,month_dmyfull,cast1_dmyfull,cast2_dmyfull,cast3_dmyfull], 
                        axis = 1 )

In [38]:
# Drop unimportant/object columns
#Movies_full = Movies_full.drop(['original_language1','production_countries1','production_countries2',
 #                               'genres1','genres2','cast1','cast2','cast3','producer1','producer2','producer3',
  #                             'director1','director2','director3','writer1'], axis = 1)
Movies_full = Movies_full.drop(['original_language1','production_countries1','production_countries2',
                                'genres1','genres2','month','cast1','cast2','cast3'], axis = 1)

In [39]:
Movies_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3507 entries, 0 to 8452
Columns: 246 entries, vote_average to 36.0
dtypes: float64(12), int64(3), uint8(231)
memory usage: 1.2 MB


In [40]:
Movies_full = Movies_full.dropna()

In [41]:
# Split into x and y for prediction
X = Movies_full.drop(['revenue'], axis = 1)
y = Movies_full['revenue']
# Separate into test and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [42]:
rf = RandomForestRegressor(criterion='mse')

#RFmodel1 = rf.fit(X_train, y_train)

#RFmodelPred1 = RFmodel1.predict(X_test)

#print(r2_score(y_test, RFmodelPred1))
#RF1meanSquaredError = mean_squared_error(y_test, RFmodelPred1)
#print("MSE:", RF1meanSquaredError)

In [43]:
# Try log
logy_test = np.log(y_test)
logy_train = np.log(y_train)

In [44]:
RFmodellog = rf.fit(X_train, logy_train)
RFmodelPredlog = RFmodellog.predict(X_test)

RFlogmeanSquaredError = mean_squared_error(logy_test, RFmodelPredlog)
print("MSE:", RFlogmeanSquaredError)
r2_score(logy_test, RFmodelPredlog) 

MSE: 0.625444805749


0.41328324885202761